# Deep Nets with TF Abstractions


# The Data

In [1]:
from sklearn.datasets import load_wine

In [2]:
wine_data = load_wine()

In [3]:
type(wine_data)

sklearn.utils.Bunch

In [4]:
wine_data.keys()

dict_keys(['target_names', 'data', 'DESCR', 'feature_names', 'target'])

In [5]:
print(wine_data.DESCR)

Wine Data Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- 1) Alcohol
 		- 2) Malic acid
 		- 3) Ash
		- 4) Alcalinity of ash  
 		- 5) Magnesium
		- 6) Total phenols
 		- 7) Flavanoids
 		- 8) Nonflavanoid phenols
 		- 9) Proanthocyanins
		- 10)Color intensity
 		- 11)Hue
 		- 12)OD280/OD315 of diluted wines
 		- 13)Proline
        	- class:
                - class_0
                - class_1
                - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash: 

In [6]:
feat_data = wine_data['data']

In [7]:
labels = wine_data['target']

### Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

### Scale the Data

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaled_x_train = scaler.fit_transform(X_train)

In [13]:
scaled_x_test = scaler.transform(X_test)

# Abstractions

## Estimator API

In [14]:
import tensorflow as tf

/Users/sridhariyer/anaconda/envs/srienv/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
from tensorflow import estimator 

In [16]:
X_train.shape

(124, 13)

In [17]:
feat_cols = [tf.feature_column.numeric_column("x", shape=[13])]

In [18]:
deep_model = estimator.DNNClassifier(hidden_units=[13,13,13],
                            feature_columns=feat_cols,
                            n_classes=3,
                            optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.01) )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_secs': 600, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_train_distribute': None, '_tf_random_seed': None, '_service': None, '_device_fn': None, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1205c8898>, '_master': '', '_save_checkpoints_steps': None, '_session_config': None, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_model_dir': '/var/folders/c5/d8zsz0ss5_54lsffwtj6ph5r0000gn/T/tmp4wj442sa', '_task_type': 'worker'}


In [19]:
input_fn = estimator.inputs.numpy_input_fn(x={'x':scaled_x_train},y=y_train,shuffle=True,batch_size=10,num_epochs=5)

In [20]:
deep_model.train(input_fn=input_fn,steps=500)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/c5/d8zsz0ss5_54lsffwtj6ph5r0000gn/T/tmp4wj442sa/model.ckpt.
INFO:tensorflow:loss = 11.235781, step = 1
INFO:tensorflow:Saving checkpoints for 62 into /var/folders/c5/d8zsz0ss5_54lsffwtj6ph5r0000gn/T/tmp4wj442sa/model.ckpt.
INFO:tensorflow:Loss for final step: 5.4830313.


In [21]:
input_fn_eval = estimator.inputs.numpy_input_fn(x={'x':scaled_x_test},shuffle=False)

In [22]:
preds = list(deep_model.predict(input_fn=input_fn_eval))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/c5/d8zsz0ss5_54lsffwtj6ph5r0000gn/T/tmp4wj442sa/model.ckpt-62
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [23]:
predictions = [p['class_ids'][0] for p in preds]

In [24]:
from sklearn.metrics import confusion_matrix,classification_report

In [25]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       1.00      0.21      0.35        19
          1       0.58      0.95      0.72        22
          2       0.93      1.00      0.96        13

avg / total       0.81      0.70      0.65        54



____________
______________

# TensorFlow Keras

### Create the Model

In [26]:
from tensorflow.contrib.keras import models

In [27]:
dnn_keras_model = models.Sequential()

### Add Layers to the model

In [28]:
from tensorflow.contrib.keras import layers

In [29]:
dnn_keras_model.add(layers.Dense(units=13,input_dim=13,activation='relu'))

In [30]:
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))
dnn_keras_model.add(layers.Dense(units=13,activation='relu'))

In [31]:
dnn_keras_model.add(layers.Dense(units=3,activation='softmax'))

### Compile the Model

In [32]:
from tensorflow.contrib.keras import losses,optimizers,metrics

In [33]:
# explore these
# losses.

In [34]:
#optimizers.

In [35]:
losses.sparse_categorical_crossentropy

<function tensorflow.python.keras.losses.sparse_categorical_crossentropy>

In [36]:
dnn_keras_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Train Model

In [37]:
dnn_keras_model.fit(scaled_x_train,y_train,epochs=50)

Epoch 1/50
124/124 [==============================] - 0s 3ms/step - loss: 1.1025 - acc: 0.3710
Epoch 2/50
124/124 [==============================] - 0s 79us/step - loss: 1.0889 - acc: 0.3629
Epoch 3/50
124/124 [==============================] - 0s 86us/step - loss: 1.0765 - acc: 0.3871
Epoch 4/50
124/124 [==============================] - 0s 82us/step - loss: 1.0665 - acc: 0.4274
Epoch 5/50
124/124 [==============================] - 0s 96us/step - loss: 1.0558 - acc: 0.4355
Epoch 6/50
124/124 [==============================] - 0s 112us/step - loss: 1.0452 - acc: 0.4758
Epoch 7/50
124/124 [==============================] - 0s 90us/step - loss: 1.0337 - acc: 0.4839
Epoch 8/50
124/124 [==============================] - 0s 89us/step - loss: 1.0203 - acc: 0.5403
Epoch 9/50
124/124 [==============================] - 0s 92us/step - loss: 1.0059 - acc: 0.5645
Epoch 10/50
124/124 [==============================] - 0s 79us/step - loss: 0.9911 - acc: 0.6290
Epoch 11/50
124/124 [==================

In [38]:
predictions = dnn_keras_model.predict_classes(scaled_x_test)

In [39]:
print(classification_report(predictions,y_test))

             precision    recall  f1-score   support

          0       1.00      0.95      0.97        20
          1       0.95      1.00      0.98        21
          2       1.00      1.00      1.00        13

avg / total       0.98      0.98      0.98        54



# Layers API

https://www.tensorflow.org/tutorials/layers

## Formating Data

In [40]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [41]:
wine_data = load_wine()
feat_data = wine_data['data']
labels = wine_data['target']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

In [43]:
scaler = MinMaxScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)
# ONE HOT ENCODED
onehot_y_train = pd.get_dummies(y_train).as_matrix()
one_hot_y_test = pd.get_dummies(y_test).as_matrix()

### Parameters

In [44]:
num_feat = 13
num_hidden1 = 13
num_hidden2 = 13
num_outputs = 3
learning_rate = 0.01

In [45]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

### Placeholder

In [46]:
X = tf.placeholder(tf.float32,shape=[None,num_feat])
y_true = tf.placeholder(tf.float32,shape=[None,3])

### Activation Function

In [47]:
actf = tf.nn.relu

### Create Layers

In [48]:
hidden1 = fully_connected(X,num_hidden1,activation_fn=actf)

In [49]:
hidden2 = fully_connected(hidden1,num_hidden2,activation_fn=actf)

In [50]:
output = fully_connected(hidden2,num_outputs)

### Loss Function

In [51]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

### Optimizer

In [52]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

### Init

In [53]:
init = tf.global_variables_initializer()

In [55]:
training_steps = 1000
with tf.Session() as sess:
    sess.run(init)
    
    for i in range(training_steps):
        sess.run(train,feed_dict={X:scaled_x_train,y_true:onehot_y_train})
        
    # Get Predictions
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    preds = tf.argmax(logits,axis=1)
    
    results = preds.eval()

In [56]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(results,y_test))

             precision    recall  f1-score   support

          0       1.00      0.61      0.76        31
          1       1.00      0.96      0.98        23
          2       0.00      0.00      0.00         0

avg / total       1.00      0.76      0.85        54



/Users/sridhariyer/anaconda/envs/srienv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
